In [1]:
import single_head
import multi_head
import copy
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.ndimage.morphology import binary_fill_holes
import sys
np.set_printoptions(threshold=sys.maxsize)
from scipy.signal import detrend
import matplotlib.gridspec as gridspec
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [2]:
# gen pickled heads
Sequence=1

In [6]:
for i in range(1,16):
    print(i)
    head = single_head.SingleHead.read_from_file(Sequence,i)
    head.full_filter()
    head.save()

1
depth filter done.
dangling removal done
color filter done.


In [4]:
# load heads
all_heads=[single_head.SingleHead.load_from_pickle(Sequence,i) for i in range (1,16)]

In [3]:
mhead=multi_head.MultiHead.load_from_pickle(Sequence)

In [31]:
mhead.save()

Saving Completed


In [29]:
# make head from first to last
def make_mhead(all_heads, first, last):
    all_heads[first].reset_positions()
    all_heads[first].reset_colors()
    all_heads[first+1].reset_positions()
    all_heads[first+1].reset_colors()   
    heads = multi_head.MultiHead.joined_heads(all_heads[first], all_heads[first+1])
    heads.links.append(multi_head.Link(left=all_heads[first+1].frame_id,right=all_heads[first].frame_id))
    for i in range(first+2,last+1):
        heads.links.append(multi_head.Link(left=all_heads[i].frame_id,right=all_heads[i-1].frame_id))
        all_heads[i].reset_positions()
        all_heads[i].reset_colors()
        heads.append(all_heads[i])        
        if i==last:
             heads.links.append(multi_head.Link(left=all_heads[first].frame_id,right=all_heads[i].frame_id))
    print ("completed")
    return heads
mhead = make_mhead(all_heads, 0, 14)
mhead.calc_keypoints()

completed


In [30]:
idx=5
def calc_connections(mhead):
    for link in mhead.links:
        link.reset()
        mhead.ransac_from_link(link)        
calc_connections(mhead)

C:\Data Science\Advanced Vision\Coursework\AV_CW\SIFT.py:89: RuntimeWarning: divide by zero encountered in double_scalars
  err = np.sqrt(np.var(dist) / (np.sum(inliers) - min_num_inliers))
C:\Data Science\Advanced Vision\Coursework\AV_CW\SIFT.py:89: RuntimeWarning: invalid value encountered in sqrt
  err = np.sqrt(np.var(dist) / (np.sum(inliers) - min_num_inliers))


In [50]:
mhead.reset_all_head_positions()
for idx in range(14):
    mhead.transform_from_link(mhead.links[idx])
    mhead.links[idx].print_short()
center, distances=mhead.left_eye_deviation()
np.mean(distances)    

2-1, matches=109, Count=68, Err=0.0058
3-2, matches=129, Count=87, Err=0.0053
4-3, matches=99, Count=58, Err=0.0048
5-4, matches=107, Count=78, Err=0.0027
6-5, matches=98, Count=65, Err=0.0030
7-6, matches=85, Count=57, Err=0.0034
8-7, matches=57, Count=34, Err=0.0030
9-8, matches=15, Count=9, Err=0.0299
10-9, matches=7, Count=5, Err=nan
11-10, matches=18, Count=12, Err=0.0075
12-11, matches=55, Count=33, Err=0.0040
13-12, matches=67, Count=36, Err=0.0035
14-13, matches=71, Count=33, Err=0.0065
15-14, matches=65, Count=37, Err=0.0112
0 172576
coordinate of the left eye: [ 0.03719031 -0.17593989 -0.02675807] in frame 0
1 172581
coordinate of the left eye: [ 0.03787625 -0.17573168 -0.02416361] in frame 1
12 169336
coordinate of the left eye: [ 0.14740933 -0.08301224  0.06097668] in frame 12
13 170000
coordinate of the left eye: [ 0.14839883 -0.0825091   0.06507843] in frame 13
14 169380
coordinate of the left eye: [ 0.14840943 -0.08538188  0.06688029] in frame 14
[ 0.10385683 -0.12051496

0.08145811445378431

In [51]:
mhead.create_spheres(0.1 ,"name")

In [246]:
mhead.reset_all_head_positions()
for idx in range(6,14):
    mhead.transform_from_link(mhead.links[idx])
    mhead.links[idx].print_short()
for idx in range(5,-1,-1):
    mhead.transform_from_link(mhead.links[idx])
    mhead.links[idx].print_short()
center, distances=mhead.left_eye_deviation()
np.mean(distances)


8-7, matches=57, Count=34, Err=0.0030



9-8, matches=15, Count=9, Err=0.0299



10-9, matches=7, Count=5, Err=nan



11-10, matches=18, Count=12, Err=0.0075



12-11, matches=55, Count=33, Err=0.0040


...


5-4, matches=107, Count=78, Err=0.0027



4-3, matches=99, Count=58, Err=0.0048



3-2, matches=129, Count=87, Err=0.0053



2-1, matches=109, Count=68, Err=0.0058
0 172576
coordinate of the left eye: [ 0.05802867 -0.1112326   0.1986909 ] in frame 0
1 172581
coordinate of the left eye: [ 0.05610711 -0.10592204  0.19808803] in frame 1
12 169336
coordinate of the left eye: [-0.10952395  0.02883137  0.22973919] in frame 12
13 170000
coordinate of the left eye: [-0.10996653  0.02697497  0.23506629] in frame 13
14 169380
coordinate of the left eye: [-0.1071064   0.01737836  0.23768785] in frame 14
[-0.04249222 -0.02879399  0.21985445]
[[ 0.10052089 -0.08243861 -0.02116355]
 [ 0.09859933 -0.07712805 -0.02176642]
 [-0.06703173  0.05762536  0.00988474]
 [-0.06747431  0.05576896  0.01521184]
 [-0.06461418  0.04617235  0.0178334 ]]
mean coordinate: [-0.04249222 -0.02879399  0.21985445]. Distance to each points: [0.13171359 0.12706039 0.08894742 0.0888502  0.08139354].


0.1035930282595802

In [115]:
def calc_distances(mhead, A,B):
    head1 = mhead.heads[mhead.head_id_from_frame_id(A)]
    head2 = mhead.heads[mhead.head_id_from_frame_id(B)]
    A=head1.xyz
    B=head2.xyz
    m = A.shape[1]

        # make points homogeneous, copy them to maintain the originals
    src = np.ones((m+1,A.shape[0]))
    dst = np.ones((m+1,B.shape[0]))
    src[:m,:] = np.copy(A.T)
    dst[:m,:] = np.copy(B.T)

    # apply the initial pose estimation

    prev_error = -1
    distances, indices = icp.nearest_neighbor(src[:m,:].T, dst[:m,:].T)
    return distances

In [198]:
def refine(mhead, A,B, offset,step, axis):
    best_count=-1
    for d in np.arange(-offset,offset,step):
        t=np.array(axis) *d
        mhead.heads[mhead.head_id_from_frame_id(A)].xyz = mhead.heads[mhead.head_id_from_frame_id(A)].xyz+t
        distances2 = calc_distances(mhead,A, B)    
        count = np.sum(distances2 <step*2)
        mean_dist=np.sqrt(np.mean(distances2**2))
        if count > best_count:
            best_t = t
            best_count = count
        print(f"{d:.3f}, {mean_dist:.4f}, {count}, {best_count}")
        mhead.heads[mhead.head_id_from_frame_id(mhead.links[idx].left)].xyz = mhead.heads[mhead.head_id_from_frame_id(mhead.links[idx].left)].xyz-t
    mhead.heads[mhead.head_id_from_frame_id(mhead.links[idx].left)].xyz = mhead.heads[mhead.head_id_from_frame_id(mhead.links[idx].left)].xyz+best_t    

def refine3d(mhead, A,B):    
    refine(mhead,A,B,0.05,0.01,[1,0,0])
    refine(mhead,A,B,0.05,0.01,[0,1,0])
    refine(mhead,A,B,0.05,0.01,[0,0,1])
    refine(mhead,A,B,0.01,0.002,[1,0,0])
    refine(mhead,A,B,0.01,0.002,[0,1,0])
    refine(mhead,A,B,0.01,0.002,[0,0,1])


In [62]:
import icp

mhead.reset_all_head_positions()
for idx in [0,1,2,3,4,5,6,7,8,9,10,11,12,13]:
    mhead.transform_from_link(mhead.links[idx])

# mhead.heads[mhead.head_id_from_frame_id(mhead.links[7].right)].paint([1,1,0])

mhead.create_spheres(0.2,name="hoho")

0, -10.0, 1914, -10.0
0, -9.0, 2160, -9.0
0, -8.0, 2474, -8.0
0, -7.0, 2850, -7.0
0, -6.0, 3205, -6.0
0, -5.0, 3686, -5.0
0, -4.0, 4167, -4.0
0, -3.0, 4502, -3.0
0, -2.0, 4758, -2.0
0, -1.0, 4888, -1.0
0, -0.0, 4907, -0.0
0, 1.0, 4907, -0.0
0, 2.0, 4907, -0.0
0, 3.0, 4907, -0.0
0, 4.0, 4907, -0.0
0, 5.0, 4907, -0.0
0, 6.0, 4907, -0.0
0, 7.0, 4907, -0.0
0, 8.0, 4907, -0.0
0, 9.0, 4907, -0.0
1, -10.0, 2697, -10.0
1, -9.0, 2994, -9.0
1, -8.0, 3278, -8.0
1, -7.0, 3608, -7.0
1, -6.0, 4017, -6.0
1, -5.0, 4298, -5.0
1, -4.0, 4523, -4.0
1, -3.0, 4658, -3.0
1, -2.0, 4795, -2.0
1, -1.0, 4876, -1.0
1, -0.0, 4912, -0.0
1, 1.0, 4912, -0.0
1, 2.0, 4912, -0.0
1, 3.0, 4912, -0.0
1, 4.0, 4912, -0.0
1, 5.0, 4912, -0.0
1, 6.0, 4912, -0.0
1, 7.0, 4912, -0.0
1, 8.0, 4912, -0.0
1, 9.0, 4912, -0.0
2, -10.0, 2008, -10.0
2, -9.0, 2210, -9.0
2, -8.0, 2478, -8.0
2, -7.0, 2804, -7.0
2, -6.0, 3229, -6.0
2, -5.0, 3704, -5.0
2, -4.0, 4192, -4.0
2, -3.0, 4571, -3.0
2, -2.0, 4811, -2.0
2, -1.0, 4943, -1.0
2, -0.0, 498

0, -10.0, 2158, -10.0
0, -9.0, 2423, -9.0
0, -8.0, 2728, -8.0
0, -7.0, 3002, -7.0
0, -6.0, 3317, -6.0
0, -5.0, 3618, -5.0
0, -4.0, 3989, -4.0
0, -3.0, 4253, -3.0
0, -2.0, 4431, -2.0
0, -1.0, 4517, -1.0
0, -0.0, 4517, -1.0
0, 1.0, 4517, -1.0
0, 2.0, 4517, -1.0
0, 3.0, 4517, -1.0
0, 4.0, 4517, -1.0
0, 5.0, 4517, -1.0
0, 6.0, 4517, -1.0
0, 7.0, 4517, -1.0
0, 8.0, 4517, -1.0
0, 9.0, 4517, -1.0
1, -10.0, 1859, -10.0
1, -9.0, 2004, -9.0
1, -8.0, 2219, -8.0
1, -7.0, 2439, -7.0
1, -6.0, 2705, -6.0
1, -5.0, 2975, -5.0
1, -4.0, 3294, -4.0
1, -3.0, 3677, -3.0
1, -2.0, 4086, -2.0
1, -1.0, 4335, -1.0
1, -0.0, 4495, -0.0
1, 1.0, 4608, 1.0
1, 2.0, 4679, 2.0
1, 3.0, 4679, 2.0
1, 4.0, 4679, 2.0
1, 5.0, 4679, 2.0
1, 6.0, 4679, 2.0
1, 7.0, 4679, 2.0
1, 8.0, 4679, 2.0
1, 9.0, 4679, 2.0
2, -10.0, 1718, -10.0
2, -9.0, 1876, -9.0
2, -8.0, 2107, -8.0
2, -7.0, 2400, -7.0
2, -6.0, 2744, -6.0
2, -5.0, 3180, -5.0
2, -4.0, 3681, -4.0
2, -3.0, 4141, -3.0
2, -2.0, 4427, -2.0
2, -1.0, 4650, -1.0
2, -0.0, 4815, -0.0
2

0, -10.0, 2120, -10.0
0, -9.0, 2315, -9.0
0, -8.0, 2573, -8.0
0, -7.0, 2915, -7.0
0, -6.0, 3299, -6.0
0, -5.0, 3724, -5.0
0, -4.0, 4156, -4.0
0, -3.0, 4551, -3.0
0, -2.0, 4772, -2.0
0, -1.0, 4878, -1.0
0, -0.0, 4887, -0.0
0, 1.0, 4887, -0.0
0, 2.0, 4887, -0.0
0, 3.0, 4887, -0.0
0, 4.0, 4887, -0.0
0, 5.0, 4887, -0.0
0, 6.0, 4887, -0.0
0, 7.0, 4887, -0.0
0, 8.0, 4887, -0.0
0, 9.0, 4887, -0.0
1, -10.0, 2862, -10.0
1, -9.0, 3079, -9.0
1, -8.0, 3301, -8.0
1, -7.0, 3539, -7.0
1, -6.0, 3765, -6.0
1, -5.0, 3987, -5.0
1, -4.0, 4259, -4.0
1, -3.0, 4481, -3.0
1, -2.0, 4585, -2.0
1, -1.0, 4722, -1.0
1, -0.0, 4797, -0.0
1, 1.0, 4806, 1.0
1, 2.0, 4806, 1.0
1, 3.0, 4806, 1.0
1, 4.0, 4806, 1.0
1, 5.0, 4806, 1.0
1, 6.0, 4806, 1.0
1, 7.0, 4806, 1.0
1, 8.0, 4806, 1.0
1, 9.0, 4806, 1.0
2, -10.0, 1782, -10.0
2, -9.0, 1930, -9.0
2, -8.0, 2154, -8.0
2, -7.0, 2428, -7.0
2, -6.0, 2779, -6.0
2, -5.0, 3226, -5.0
2, -4.0, 3775, -4.0
2, -3.0, 4249, -3.0
2, -2.0, 4626, -2.0
2, -1.0, 4797, -1.0
2, -0.0, 4853, -0.0
2

0, -10.0, 1902, -10.0
0, -9.0, 2127, -9.0
0, -8.0, 2358, -8.0
0, -7.0, 2692, -7.0
0, -6.0, 3024, -6.0
0, -5.0, 3330, -5.0
0, -4.0, 3679, -4.0
0, -3.0, 4094, -3.0
0, -2.0, 4417, -2.0
0, -1.0, 4657, -1.0
0, -0.0, 4790, -0.0
0, 1.0, 4790, -0.0
0, 2.0, 4790, -0.0
0, 3.0, 4790, -0.0
0, 4.0, 4790, -0.0
0, 5.0, 4790, -0.0
0, 6.0, 4790, -0.0
0, 7.0, 4790, -0.0
0, 8.0, 4790, -0.0
0, 9.0, 4790, -0.0
1, -10.0, 3210, -10.0
1, -9.0, 3380, -9.0
1, -8.0, 3584, -8.0
1, -7.0, 3791, -7.0
1, -6.0, 4044, -6.0
1, -5.0, 4333, -5.0
1, -4.0, 4463, -4.0
1, -3.0, 4555, -3.0
1, -2.0, 4661, -2.0
1, -1.0, 4702, -1.0
1, -0.0, 4702, -1.0
1, 1.0, 4702, -1.0
1, 2.0, 4702, -1.0
1, 3.0, 4702, -1.0
1, 4.0, 4702, -1.0
1, 5.0, 4702, -1.0
1, 6.0, 4702, -1.0
1, 7.0, 4702, -1.0
1, 8.0, 4702, -1.0
1, 9.0, 4702, -1.0
2, -10.0, 1935, -10.0
2, -9.0, 2152, -9.0
2, -8.0, 2393, -8.0
2, -7.0, 2699, -7.0
2, -6.0, 3074, -6.0
2, -5.0, 3496, -5.0
2, -4.0, 3889, -4.0
2, -3.0, 4266, -3.0
2, -2.0, 4575, -2.0
2, -1.0, 4762, -1.0
2, -0.0, 478

0, -10.0, 2163, -10.0
0, -9.0, 2289, -9.0
0, -8.0, 2458, -8.0
0, -7.0, 2750, -7.0
0, -6.0, 3087, -6.0
0, -5.0, 3447, -5.0
0, -4.0, 3784, -4.0
0, -3.0, 4197, -3.0
0, -2.0, 4448, -2.0
0, -1.0, 4498, -1.0
0, -0.0, 4498, -1.0
0, 1.0, 4498, -1.0
0, 2.0, 4498, -1.0
0, 3.0, 4498, -1.0
0, 4.0, 4498, -1.0
0, 5.0, 4498, -1.0
0, 6.0, 4498, -1.0
0, 7.0, 4498, -1.0
0, 8.0, 4498, -1.0
0, 9.0, 4498, -1.0
1, -10.0, 2406, -10.0
1, -9.0, 2605, -9.0
1, -8.0, 2846, -8.0
1, -7.0, 3156, -7.0
1, -6.0, 3414, -6.0
1, -5.0, 3613, -5.0
1, -4.0, 3836, -4.0
1, -3.0, 4107, -3.0
1, -2.0, 4235, -2.0
1, -1.0, 4324, -1.0
1, -0.0, 4365, -0.0
1, 1.0, 4374, 1.0
1, 2.0, 4374, 1.0
1, 3.0, 4374, 1.0
1, 4.0, 4374, 1.0
1, 5.0, 4374, 1.0
1, 6.0, 4374, 1.0
1, 7.0, 4374, 1.0
1, 8.0, 4374, 1.0
1, 9.0, 4374, 1.0
2, -10.0, 1688, -10.0
2, -9.0, 1873, -9.0
2, -8.0, 2070, -8.0
2, -7.0, 2316, -7.0
2, -6.0, 2623, -6.0
2, -5.0, 3079, -5.0
2, -4.0, 3520, -4.0
2, -3.0, 3911, -3.0
2, -2.0, 4171, -2.0
2, -1.0, 4319, -1.0
2, -0.0, 4323, -0.0
2

...

0, -10.0, 1346, -10.0
0, -9.0, 1472, -9.0
0, -8.0, 1640, -8.0
0, -7.0, 1842, -7.0
0, -6.0, 2086, -6.0
0, -5.0, 2327, -5.0
0, -4.0, 2576, -4.0
0, -3.0, 2788, -3.0
0, -2.0, 2940, -2.0
0, -1.0, 3066, -1.0
0, -0.0, 3157, -0.0
0, 1.0, 3208, 1.0
0, 2.0, 3208, 1.0
0, 3.0, 3208, 1.0
0, 4.0, 3208, 1.0
0, 5.0, 3208, 1.0
0, 6.0, 3208, 1.0
0, 7.0, 3208, 1.0
0, 8.0, 3208, 1.0
0, 9.0, 3208, 1.0
1, -10.0, 1716, -10.0
1, -9.0, 1891, -9.0
1, -8.0, 2049, -8.0
1, -7.0, 2257, -7.0
1, -6.0, 2459, -6.0
1, -5.0, 2690, -5.0
1, -4.0, 2871, -4.0
1, -3.0, 3027, -3.0
1, -2.0, 3138, -2.0
1, -1.0, 3168, -1.0
1, -0.0, 3172, -0.0
1, 1.0, 3172, -0.0
1, 2.0, 3172, -0.0
1, 3.0, 3172, -0.0
1, 4.0, 3172, -0.0
1, 5.0, 3172, -0.0
1, 6.0, 3172, -0.0
1, 7.0, 3172, -0.0
1, 8.0, 3172, -0.0
1, 9.0, 3172, -0.0
2, -10.0, 873, -10.0
2, -9.0, 1061, -9.0
2, -8.0, 1300, -8.0
2, -7.0, 1634, -7.0
2, -6.0, 2029, -6.0
2, -5.0, 2347, -5.0
2, -4.0, 2620, -4.0
2, -3.0, 2874, -3.0
2, -2.0, 3117, -2.0
2, -1.0, 3226, -1.0
2, -0.0, 3241, -0.0
2,

0, -10.0, 2039, -10.0
0, -9.0, 2224, -9.0
0, -8.0, 2443, -8.0
0, -7.0, 2746, -7.0
0, -6.0, 3023, -6.0
0, -5.0, 3363, -5.0
0, -4.0, 3713, -4.0
0, -3.0, 3961, -3.0
0, -2.0, 4101, -2.0
0, -1.0, 4162, -1.0
0, -0.0, 4167, -0.0
0, 1.0, 4167, -0.0
0, 2.0, 4167, -0.0
0, 3.0, 4167, -0.0
0, 4.0, 4167, -0.0
0, 5.0, 4167, -0.0
0, 6.0, 4167, -0.0
0, 7.0, 4167, -0.0
0, 8.0, 4167, -0.0
0, 9.0, 4167, -0.0
1, -10.0, 2500, -10.0
1, -9.0, 2727, -9.0
1, -8.0, 2959, -8.0
1, -7.0, 3243, -7.0
1, -6.0, 3461, -6.0
1, -5.0, 3675, -5.0
1, -4.0, 3848, -4.0
1, -3.0, 4000, -3.0
1, -2.0, 4123, -2.0
1, -1.0, 4218, -1.0
1, -0.0, 4273, -0.0
1, 1.0, 4273, -0.0
1, 2.0, 4273, -0.0
1, 3.0, 4273, -0.0
1, 4.0, 4273, -0.0
1, 5.0, 4273, -0.0
1, 6.0, 4273, -0.0
1, 7.0, 4273, -0.0
1, 8.0, 4273, -0.0
1, 9.0, 4273, -0.0
2, -10.0, 1582, -10.0
2, -9.0, 1704, -9.0
2, -8.0, 1869, -8.0
2, -7.0, 2032, -7.0
2, -6.0, 2301, -6.0
2, -5.0, 2597, -5.0
2, -4.0, 2963, -4.0
2, -3.0, 3407, -3.0
2, -2.0, 3824, -2.0
2, -1.0, 4095, -1.0
2, -0.0, 418

0, -10.0, 2439, -10.0
0, -9.0, 2683, -9.0
0, -8.0, 2887, -8.0
0, -7.0, 3047, -7.0
0, -6.0, 3261, -6.0
0, -5.0, 3426, -5.0
0, -4.0, 3620, -4.0
0, -3.0, 3808, -3.0
0, -2.0, 3974, -2.0
0, -1.0, 4028, -1.0
0, -0.0, 4028, -1.0
0, 1.0, 4028, -1.0
0, 2.0, 4028, -1.0
0, 3.0, 4028, -1.0
0, 4.0, 4028, -1.0
0, 5.0, 4028, -1.0
0, 6.0, 4028, -1.0
0, 7.0, 4028, -1.0
0, 8.0, 4028, -1.0
0, 9.0, 4028, -1.0
1, -10.0, 2212, -10.0
1, -9.0, 2424, -9.0
1, -8.0, 2690, -8.0
1, -7.0, 2949, -7.0
1, -6.0, 3204, -6.0
1, -5.0, 3420, -5.0
1, -4.0, 3634, -4.0
1, -3.0, 3839, -3.0
1, -2.0, 3986, -2.0
1, -1.0, 4064, -1.0
1, -0.0, 4064, -1.0
1, 1.0, 4064, -1.0
1, 2.0, 4064, -1.0
1, 3.0, 4064, -1.0
1, 4.0, 4064, -1.0
1, 5.0, 4064, -1.0
1, 6.0, 4064, -1.0
1, 7.0, 4064, -1.0
1, 8.0, 4064, -1.0
1, 9.0, 4064, -1.0
2, -10.0, 1516, -10.0
2, -9.0, 1659, -9.0
2, -8.0, 1833, -8.0
2, -7.0, 2033, -7.0
2, -6.0, 2277, -6.0
2, -5.0, 2523, -5.0
2, -4.0, 2776, -4.0
2, -3.0, 3060, -3.0
2, -2.0, 3413, -2.0
2, -1.0, 3824, -1.0
2, -0.0, 409

0, -10.0, 1650, -10.0
0, -9.0, 1822, -9.0
0, -8.0, 2024, -8.0
0, -7.0, 2242, -7.0
0, -6.0, 2522, -6.0
0, -5.0, 2875, -5.0
0, -4.0, 3178, -4.0
0, -3.0, 3521, -3.0
0, -2.0, 3757, -2.0
0, -1.0, 3918, -1.0
0, -0.0, 4025, -0.0
0, 1.0, 4025, -0.0
0, 2.0, 4025, -0.0
0, 3.0, 4025, -0.0
0, 4.0, 4025, -0.0
0, 5.0, 4025, -0.0
0, 6.0, 4025, -0.0
0, 7.0, 4025, -0.0
0, 8.0, 4025, -0.0
0, 9.0, 4025, -0.0
1, -10.0, 2788, -10.0
1, -9.0, 2935, -9.0
1, -8.0, 3135, -8.0
1, -7.0, 3406, -7.0
1, -6.0, 3673, -6.0
1, -5.0, 4035, -5.0
1, -4.0, 4215, -4.0
1, -3.0, 4341, -3.0
1, -2.0, 4363, -2.0
1, -1.0, 4363, -2.0
1, -0.0, 4363, -2.0
1, 1.0, 4363, -2.0
1, 2.0, 4363, -2.0
1, 3.0, 4363, -2.0
1, 4.0, 4363, -2.0
1, 5.0, 4363, -2.0
1, 6.0, 4363, -2.0
1, 7.0, 4363, -2.0
1, 8.0, 4363, -2.0
1, 9.0, 4363, -2.0
2, -10.0, 1776, -10.0
2, -9.0, 1970, -9.0
2, -8.0, 2231, -8.0
2, -7.0, 2554, -7.0
2, -6.0, 2930, -6.0
2, -5.0, 3374, -5.0
2, -4.0, 3788, -4.0
2, -3.0, 4067, -3.0
2, -2.0, 4209, -2.0
2, -1.0, 4282, -1.0
2, -0.0, 434

0, -10.0, 1919, -10.0
0, -9.0, 2132, -9.0
0, -8.0, 2402, -8.0
0, -7.0, 2708, -7.0
0, -6.0, 3073, -6.0
0, -5.0, 3401, -5.0
0, -4.0, 3649, -4.0
0, -3.0, 3861, -3.0
0, -2.0, 4127, -2.0
0, -1.0, 4386, -1.0
0, -0.0, 4551, -0.0
0, 1.0, 4551, -0.0
0, 2.0, 4551, -0.0
0, 3.0, 4551, -0.0
0, 4.0, 4551, -0.0
0, 5.0, 4551, -0.0
0, 6.0, 4551, -0.0
0, 7.0, 4551, -0.0
0, 8.0, 4551, -0.0
0, 9.0, 4551, -0.0
1, -10.0, 2764, -10.0
1, -9.0, 2958, -9.0
1, -8.0, 3199, -8.0
1, -7.0, 3396, -7.0
1, -6.0, 3638, -6.0
1, -5.0, 3841, -5.0
1, -4.0, 4065, -4.0
1, -3.0, 4295, -3.0
1, -2.0, 4410, -2.0
1, -1.0, 4500, -1.0
1, -0.0, 4539, -0.0
1, 1.0, 4539, -0.0
1, 2.0, 4539, -0.0
1, 3.0, 4539, -0.0
1, 4.0, 4539, -0.0
1, 5.0, 4539, -0.0
1, 6.0, 4539, -0.0
1, 7.0, 4539, -0.0
1, 8.0, 4539, -0.0
1, 9.0, 4539, -0.0
2, -10.0, 1837, -10.0
2, -9.0, 2038, -9.0
2, -8.0, 2335, -8.0
2, -7.0, 2654, -7.0
2, -6.0, 3001, -6.0
2, -5.0, 3473, -5.0
2, -4.0, 4012, -4.0
2, -3.0, 4463, -3.0
2, -2.0, 4651, -2.0
2, -1.0, 4726, -1.0
2, -0.0, 472

In [63]:
center, distances=mhead.left_eye_deviation()
np.mean(distances)

0 172576
coordinate of the left eye: [ 0.03719031 -0.17593989 -0.02675807] in frame 0
1 172581
coordinate of the left eye: [ 0.04758514 -0.17892697 -0.02519165] in frame 1
12 169336
coordinate of the left eye: [ 0.02766827 -0.19752245  0.00875789] in frame 12
13 170000
coordinate of the left eye: [ 0.03506904 -0.19499765  0.00161856] in frame 13
14 169380
coordinate of the left eye: [ 0.03352019 -0.19212895 -0.00254084] in frame 14
[ 0.03620659 -0.18790318 -0.00882282]
[[ 0.00098372  0.01196329 -0.01793525]
 [ 0.01137855  0.00897621 -0.01636883]
 [-0.00853832 -0.00961927  0.01758071]
 [-0.00113755 -0.00709447  0.01044138]
 [-0.0026864  -0.00422577  0.00628198]]
mean coordinate: [ 0.03620659 -0.18790318 -0.00882282]. Distance to each points: [0.0215815  0.0218628  0.02178336 0.0126747  0.0080335 ].


0.017187172111814947

In [90]:
mhead.links[7].left

9

In [170]:
only_first_n=15
mhead.reset_all_head_positions()
link_index,err=mhead.get_next_unpositioned_link()
positioned_head_count=0
joined_heads=set() 
while(not link_index is None) and (positioned_head_count<only_first_n or only_first_n==-1):
    mhead.links[link_index].print_short()
    joined_heads.add(mhead.links[link_index].left)
    joined_heads.add(mhead.links[link_index].right)
    mhead.heads[mhead.links[link_index].right-1].background_color=np.asarray([0,0,1])
    mhead.heads[mhead.links[link_index].left-1].background_color=np.asarray([0,0,1]) 
    foo= mhead.transform_from_link(mhead.links[link_index])   
    link_index,err=mhead.get_next_unpositioned_link()
    positioned_head_count=max(positioned_head_count+1,2)
joined_heads=sorted(joined_heads)
print(joined_heads)
no_heads= len(joined_heads)
joined_heads=[str(h) for h in joined_heads]
name= f"Seq_{mhead.heads[0].sequence_id}_frames_{'_'.join(joined_heads)}"
mhead.create_spheres(min(2/max(no_heads,0.2),1),name=name)
center, distances=mhead.left_eye_deviation()
np.mean(distances)

any_head_positioned False
1-15, matches=126, Count=90, Err=0.0026
-0.050, 0.0219, 32625, 32625
-0.040, 0.0176, 37273, 37273
-0.030, 0.0135, 44998, 44998
-0.020, 0.0098, 50838, 50838
-0.010, 0.0069, 52385, 52385
0.000, 0.0058, 52237, 52385
0.010, 0.0071, 51890, 52385
0.020, 0.0099, 50715, 52385
0.030, 0.0133, 46106, 52385
0.040, 0.0170, 40463, 52385
-0.050, 0.0285, 16772, 16772
-0.040, 0.0231, 22155, 22155
-0.030, 0.0179, 36106, 36106
-0.020, 0.0131, 47859, 47859
-0.010, 0.0090, 52049, 52049
0.000, 0.0069, 52385, 52385
0.010, 0.0080, 52050, 52385
0.020, 0.0115, 48665, 52385
0.030, 0.0160, 41009, 52385
0.040, 0.0210, 31269, 52385
-0.050, 0.0398, 3925, 3925
-0.040, 0.0321, 5895, 5895
-0.030, 0.0246, 14131, 14131
-0.020, 0.0175, 38258, 38258
-0.010, 0.0110, 50558, 50558
0.000, 0.0069, 52385, 52385
0.010, 0.0087, 51463, 52385
0.020, 0.0143, 45281, 52385
0.030, 0.0208, 28712, 52385
0.040, 0.0276, 15410, 52385
-0.010, 0.0098, 16883, 16883
-0.008, 0.0091, 18096, 18096
-0.006, 0.0085, 19422, 19

0.006, 0.0060, 20735, 20735
0.008, 0.0055, 24962, 24962
-0.010, 0.0079, 12117, 12117
-0.008, 0.0072, 13692, 13692
-0.006, 0.0066, 15813, 15813
-0.004, 0.0061, 18507, 18507
-0.002, 0.0058, 21912, 21912
0.000, 0.0055, 24962, 24962
0.002, 0.0054, 26724, 26724
0.004, 0.0055, 27362, 27362
0.006, 0.0057, 26715, 27362
0.008, 0.0060, 24846, 27362
-0.010, 0.0087, 10852, 10852
-0.008, 0.0079, 13701, 13701
-0.006, 0.0071, 17343, 17343
-0.004, 0.0064, 21389, 21389
-0.002, 0.0059, 24655, 24655
0.000, 0.0055, 27362, 27362
0.002, 0.0052, 29130, 29130
0.004, 0.0052, 29244, 29244
0.006, 0.0054, 27470, 29244
0.008, 0.0058, 24642, 29244
any_head_positioned True
6-5, matches=98, Count=65, Err=0.0030
-0.050, 0.0412, 4553, 4553
-0.040, 0.0336, 7617, 7617
-0.030, 0.0263, 12107, 12107
-0.020, 0.0194, 22173, 22173
-0.010, 0.0134, 31750, 31750
0.000, 0.0096, 34216, 34216
0.010, 0.0104, 34211, 34216
0.020, 0.0147, 32691, 34216
0.030, 0.0204, 23842, 34216
0.040, 0.0265, 13961, 34216
-0.050, 0.0279, 15606, 15606
-

0.030, 0.0179, 34651, 42944
0.040, 0.0222, 28442, 42944
-0.050, 0.0372, 8462, 8462
-0.040, 0.0307, 12518, 12518
-0.030, 0.0246, 18954, 18954
-0.020, 0.0189, 28486, 28486
-0.010, 0.0142, 39312, 39312
0.000, 0.0114, 42944, 42944
0.010, 0.0117, 41832, 42944
0.020, 0.0148, 38249, 42944
0.030, 0.0194, 32320, 42944
0.040, 0.0247, 25632, 42944
-0.010, 0.0125, 13791, 13791
-0.008, 0.0121, 14460, 14460
-0.006, 0.0117, 15397, 15397
-0.004, 0.0115, 15498, 15498
-0.002, 0.0114, 14634, 15498
0.000, 0.0114, 13246, 15498
0.002, 0.0115, 11362, 15498
0.004, 0.0117, 9344, 15498
0.006, 0.0121, 7765, 15498
0.008, 0.0126, 6622, 15498
-0.010, 0.0129, 11427, 11427
-0.008, 0.0125, 12019, 12019
-0.006, 0.0121, 12682, 12682
-0.004, 0.0118, 13585, 13585
-0.002, 0.0116, 14625, 14625
0.000, 0.0115, 15498, 15498
0.002, 0.0115, 16242, 16242
0.004, 0.0115, 16918, 16918
0.006, 0.0117, 17446, 17446
0.008, 0.0119, 17930, 17930
-0.010, 0.0134, 8731, 8731
-0.008, 0.0129, 10080, 10080
-0.006, 0.0124, 12127, 12127
-0.004, 0

any_head_positioned True
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
0 172576
coordinate of the left eye: [ 0.02132567 -0.1780084  -0.01913775] in frame 0
1 172581
coordinate of the left eye: [ 0.02186936 -0.17588632 -0.0187414 ] in frame 1
12 169336
coordinate of the left eye: [ 0.01811048 -0.16311562  0.00041593] in frame 12
13 170000
coordinate of the left eye: [ 0.03420423 -0.16902184 -0.01630774] in frame 13
14 169380
coordinate of the left eye: [ 0.03902839 -0.17648688 -0.00631632] in frame 14
[ 0.02690762 -0.17250381 -0.01201745]
[[-0.00558196 -0.00550458 -0.0071203 ]
 [-0.00503827 -0.00338251 -0.00672394]
 [-0.00879715  0.00938819  0.01243338]
 [ 0.00729661  0.00348197 -0.00429028]
 [ 0.01212076 -0.00398307  0.00570114]]
mean coordinate: [ 0.02690762 -0.17250381 -0.01201745]. Distance to each points: [0.01059044 0.00905742 0.01789181 0.00915266 0.01397429].


0.012133322948721065

In [191]:
mhead.create_spheres(min(2/max(no_heads,0.2),1),name=name)
mhead.save_spheres("boe")

IndexError: boolean index did not match indexed array along dimension 0; dimension is 50834 but corresponding boolean dimension is 52170

In [117]:
center, distances=mhead.left_eye_deviation(1)
np.mean(distances)

0 153974
coordinate of the left eye: [-0.04270943 -0.22482576 -0.0096941 ] in frame 0
1 150143
coordinate of the left eye: [-0.046029   -0.23158732 -0.01141736] in frame 1
2 150124
coordinate of the left eye: [-0.07649181 -0.22327776  0.02547467] in frame 2
13 147576
coordinate of the left eye: [-0.01553875 -0.23271609 -0.01817739] in frame 13
14 150775
coordinate of the left eye: [-0.04775565 -0.22426942 -0.00866981] in frame 14
[-0.04570493 -0.22733527 -0.0044968 ]
[[ 0.0029955   0.00250951 -0.0051973 ]
 [-0.00032407 -0.00425205 -0.00692056]
 [-0.03078688  0.00405751  0.02997147]
 [ 0.03016618 -0.00538082 -0.01368059]
 [-0.00205072  0.00306585 -0.00417301]]
mean coordinate: [-0.04570493 -0.22733527 -0.0044968 ]. Distance to each points: [0.00650251 0.00812891 0.04315767 0.03355756 0.00556946].


0.019383222972255216

In [ ]:
for i, link in enumerate(mhead.links):
    print (i)
    link.print_short()

In [ ]:
    link_index,err=mhead.get_next_unpositioned_link()
    positioned_head_count=max(positioned_head_count+1,2)
joined_heads=sorted(joined_heads)
print(joined_heads)
no_heads= len(joined_heads)
joined_heads=[str(h) for h in joined_heads]
name= f"Seq_{mhead.heads[0].sequence_id}_frames_{'_'.join(joined_heads)}"
mhead.create_spheres(min(2/max(no_heads,0.2),1),name=name)

In [ ]:
foo= mhead.transform_from_link(mhead.links[link_index])

In [ ]:
mhead.reset_all_head_positions()
mhead.transform_from_link(mhead.links[0])
mhead.transform_from_link(mhead.links[1])
mhead.create_spheres(0.3)


In [123]:
import pandas as pd
df=pd.DataFrame(columns=['Right', 'Left',  'Matches', 'Inliers','Variance'])
for link in mhead.links:
    df = df.append({'Right' : int(link.right) , 'Left' : int(link.left) ,  'Variance' : link.err, 'Inliers' : sum(link.inliers), 'Matches' : len(link.matches)} , ignore_index=True)
df[['Right', 'Left',  'Matches', 'Inliers']] = df[['Right', 'Left',  'Matches', 'Inliers']].astype(int)
df=df.rename(columns={'Variance':"Transformation MSE"})
print(df.to_latex(index=False))

\begin{tabular}{rrrrr}
\toprule
 Right &  Left &  Matches &  Inliers &    \textbackslash sigma \\
\midrule
     1 &     2 &      111 &       78 &  0.002994 \\
     2 &     3 &      132 &       86 &  0.003627 \\
     3 &     4 &       91 &       54 &  0.005317 \\
     4 &     5 &      105 &       80 &  0.002748 \\
     5 &     6 &       94 &       67 &  0.005150 \\
     6 &     7 &       83 &       53 &  0.005607 \\
     7 &     8 &       62 &       39 &  0.003021 \\
     8 &     9 &       16 &       12 &  0.013064 \\
     9 &    10 &        8 &        5 &       NaN \\
    10 &    11 &       22 &       19 &  0.002995 \\
    11 &    12 &       55 &       27 &  0.007341 \\
    12 &    13 &       66 &       38 &  0.003067 \\
    13 &    14 &       78 &       33 &  0.007001 \\
    14 &    15 &       73 &       43 &  0.007859 \\
    15 &     1 &      124 &       91 &  0.002372 \\
\bottomrule
\end{tabular}



In [119]:
df

,Right,Left,Matches,Inliers,Variance
0,1,2,111,78,0.002994
1,2,3,132,86,0.003627
2,3,4,91,54,0.005317
3,4,5,105,80,0.002748
4,5,6,94,67,0.005150
5,6,7,83,53,0.005607
6,7,8,62,39,0.003021
7,8,9,16,12,0.013064
8,9,10,8,5,NaN
9,10,11,22,19,0.002995


In [ ]:
mhead.links[0].left

In [ ]:
mhead.create_mesh(f"{Sequence}_mesh")

In [ ]:
nrows,ncols=3,2
screenwidth=12
plt.figure(figsize = (screenwidth,3/4*nrows*screenwidth/ncols))
gs1 = gridspec.GridSpec(nrows, ncols)
gs1.update(wspace=0.1, hspace=0.1)
for row in range(nrows):
    for col in range(ncols):
        idx=ncols*row+col
        ax=plt.subplot(gs1[idx])
        mhead.heads[idx].background_color=np.array([0,0,1])
        img= mhead.heads[idx].get_filtered_image()
        ax.imshow(img)
        ax.set_axis_off()        
        ax.text(x=10, y=463 , s=f"S_{Sequence}:{idx+1}/15",horizontalalignment='left',
                              verticalalignment='bottom',  bbox=dict(facecolor='white', alpha=1))        
        del ax
plt.savefig(f"2d_filtered_seq_{Sequence}")
plt.show()

In [ ]:
sequence_ids=[0,1,2]
head_ids=[0,5,7]
abcd="abcd"
nrows,ncols=3,4
screenwidth=12
plt.figure(figsize = (screenwidth,3/4*nrows*screenwidth/ncols))
gs1 = gridspec.GridSpec(nrows, ncols)
gs1.update(wspace=0.1, hspace=0.1)
for row in range(nrows):
    head = sequence_ids[row]
    head = sequence_ids[row]
    sequence= sequence_ids[row]
    head_idx=head_ids[row]
    for col in range(ncols):
#         print (sequence, head_idx, col)
        head=mheads[sequence].heads[head_idx]
        if col==0:
            img= head.twoD_image            
            
        elif col==1:
            head.reset_filters()
            head.filter_depth(1)
            head.background_color=np.array([0,0,1])
            img= head.get_filtered_image()
        elif col==2:
            head.remove_dangling()
            img= head.get_filtered_image()
        elif col==3:
            head.remove_background_color()
            head.background_color=np.array([0,0,1])
            img= head.get_filtered_image()
        idx=ncols*row+col
        ax=plt.subplot(gs1[idx])
        ax.imshow(img)
        ax.set_axis_off()        
        if col==0:
            ax.text(x=20, y=20 , s=f"S_{sequence+1}:{head_idx+1}/15",horizontalalignment='left',
                                  verticalalignment='top',  bbox=dict(facecolor='white', alpha=1))        

        ax.text(x=20, y=453 , s=f"{abcd[col]}",horizontalalignment='left',
                              verticalalignment='bottom',  bbox=dict(facecolor='white', alpha=1))        
        
        del ax
        
plt.savefig(f"2d_full_seq_{Sequence}")
plt.show()

In [ ]:
linkindex=6
mhead.reset_all_head_positions()
foo= mhead.transform_from_link(mhead.links[linkindex])
mhead.links[linkindex].print_short()
mhead.create_spheres(0.4)

In [ ]:
import glob

In [ ]:
glob.glob("images\Seq_1_frames_SIFT*.png")

In [65]:
mhead.create_mesh("hi")


completed
